### Создание таблиц
~~~sql
create table transaction_2 (
  transaction_id int4 primary key
  ,product_id int4
  ,customer_id int4
  ,transaction_date varchar(30)
  ,online_order varchar(30)
  ,order_status varchar(30)
  ,brand varchar(30)
  ,product_line varchar(30)
  ,product_class varchar(30)
  ,product_size varchar(30)
  ,list_price float4
  ,standard_cost float4
)

create table customer_2 (
  customer_id int4 primary key
  ,first_name varchar(50)
  ,last_name varchar(50)
  ,gender varchar(30)
  ,DOB varchar(50)
  ,job_title varchar(50)
  ,job_industry_category varchar(50)
  ,wealth_segment varchar(50)
  ,deceased_indicator varchar(50)
  ,owns_car varchar(30)
  ,address varchar(50)
  ,postcode varchar(30)
  ,state varchar(30)
  ,country varchar(30)
  ,property_valuation int4
)
~~~

### Задание 1
Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. 
~~~sql 
select job_industry_category, count(*) as job_count
from customer_2 c 
group by job_industry_category 
order by job_count desc
~~~

<img src=img/HW3_1.JPG width=300px height=30%>

### Задание 2
Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.
~~~sql
select extract(month from transaction_date::date) as month, 
    job_industry_category, sum(list_price)
from transaction_2 t 
join customer_2 c on t.customer_id = c.customer_id 
group by month, job_industry_category 
order by month, job_industry_category 
~~~

<img src=img/HW3_2.JPG width=300px height=30%>

### Задание 3
Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.
~~~sql
select brand, count(*)
from customer_2 c 
join transaction_2 t on c.customer_id = t.customer_id 
where job_industry_category = 'IT' and online_order = 'True' and order_status = 'Approved'
group by brand
~~~

<img src=img/HW3_3.JPG width=200px height=30%>

### Задание 4
Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества транзакций. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.
~~~sql
select c.customer_id, sum(list_price), max(list_price), min(list_price), count(*)
from transaction_2 t 
join customer_2 c on c.customer_id = t.customer_id 
group by c.customer_id 
order by sum(list_price) desc, count(*) desc
~~~

<img src=img/HW3_41.JPG width=400px height=30%>

~~~sql
select c.customer_id
	,sum(list_price) over(partition by c.customer_id) as sum_transaction
	,max(list_price) over(partition by c.customer_id) as max_transaction
	,min(list_price) over(partition by c.customer_id) as min_transaction
	,count(list_price) over(partition by c.customer_id) as count_transaction
from transaction_2 t 
join customer_2 c on c.customer_id = t.customer_id 
order by sum_transaction desc, count_transaction desc
~~~

<img src=img/HW3_42.JPG width=500px height=30%>

### Задание 5
Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.
~~~sql
with aa as(
	select c.customer_id, first_name, last_name, sum(list_price) as transaction_sum
	from customer_2 c 
	join transaction_2 t on c.customer_id = t.customer_id
	group by c.customer_id
)
select first_name, last_name, transaction_sum as sum
from aa
where transaction_sum = (select max(transaction_sum) from aa)
union
select first_name, last_name, transaction_sum
from aa
where transaction_sum = (select min(transaction_sum) from aa)
~~~

<img src=img/HW3_5.JPG width=300px height=30%>

### Задание 6
Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. 
~~~sql
select * 
from (select customer_id, transaction_id, transaction_date::date,
             rank() over(partition by customer_id order by transaction_date::date) as first_transaction 
      from transaction_2 t) 
where first_transaction=1
~~~

<img src=img/HW3_6.JPG width=500px height=30%>

### Задание 7
Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) 
~~~sql
with aa as(
	select t.customer_id, transaction_date::date, first_name, last_name, job_title,
		lag(transaction_date) over (partition by t.customer_id 
			order by transaction_date::date) as previous_date,
		lead(transaction_date) over (partition by t.customer_id) as next_date
	from transaction_2 t  
	join customer_2 c on c.customer_id = t.customer_id 
)
select first_name, last_name, job_title,
	(next_date::date - previous_date::date) as date_difference
from aa
where next_date::date - previous_date::date = 
	(select max(next_date::date - previous_date::date) from aa)
~~~

<img src=img/HW3_7.JPG width=400px height=30%>